In [1]:
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import StaleElementReferenceException

In [2]:
browser = webdriver.Chrome(ChromeDriverManager().install())

C:\Users\Ritesh\AppData\Local\Temp\ipykernel_9720\893479022.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(ChromeDriverManager().install())


In [3]:
browser.get('https://www.flipkart.com/')
browser.maximize_window()
wait = WebDriverWait(browser, 120)

In [4]:
from selenium.common.exceptions import NoSuchElementException

X_btn = None
try:
    X_btn = browser.find_element(By.XPATH, "(//button[@class='_2KpZ6l _2doB4z'])")
except NoSuchElementException:
    print("The button was not found.")
    
if X_btn:
    X_btn.click()

The button was not found.


In [5]:
search_text_lower = "search for products, brands and more"
search_text_upper = "Search for Products, Brands and More"

xpath_expression = f"//input[contains(translate(@placeholder, 'ABCDEFGHIJKLMNOPQRSTUVWXYZ', 'abcdefghijklmnopqrstuvwxyz'), '{search_text_lower}') or contains(translate(@placeholder, 'abcdefghijklmnopqrstuvwxyz', 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'), '{search_text_upper}')]"

input_search = browser.find_element(By.XPATH, xpath_expression)
search_button = browser.find_element(By.XPATH, "(//button[@type='submit'])")

In [6]:
input_search.send_keys('bags')
search_button.click()

In [7]:
products = []
for i in range(20):
    print('Scraping page', i+1)
    
    names = browser.find_elements(By.XPATH, "//a[@class='IRpwTa']")
    prices = browser.find_elements(By.XPATH, "//div[@class='_30jeq3']")
    links = browser.find_elements(By.XPATH, "//a[@class='IRpwTa']")
    
    for n, p, l in zip(names, prices, links):
        try:
            name = n.text
            price = p.text
            price = price.replace('₹', '').replace(',','')
            link = l.get_attribute('href')
            
            products.append({'Prod_name': name, 'Prod_price': price, 'Prod_link': link})
        except StaleElementReferenceException:
            continue
            
    next_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//span[contains(text(), 'Next')]")))
    action = ActionChains(browser)
    action.move_to_element(next_button).click().perform()

Scraping page 1
Scraping page 2
Scraping page 3
Scraping page 4
Scraping page 5
Scraping page 6
Scraping page 7
Scraping page 8
Scraping page 9
Scraping page 10
Scraping page 11
Scraping page 12
Scraping page 13
Scraping page 14
Scraping page 15
Scraping page 16
Scraping page 17
Scraping page 18
Scraping page 19
Scraping page 20


In [8]:
print(len(products))
print(products[:5])

241
[{'Prod_name': 'Medium 25 L Backpack DAYPACK Bags Backpack for daily us...', 'Prod_price': '279', 'Prod_link': 'https://www.flipkart.com/zcraft-daypack-bags-backpack-daily-use-library-office-outdoor-hiking-25-l/p/itme85ed9dd77fbf?pid=BKPGNGGZ3FCRSBAB&lid=LSTBKPGNGGZ3FCRSBABW1ARSV&marketplace=FLIPKART&q=bags&store=reh%2F4d7&srno=s_1_1&otracker=search&otracker1=search&fm=organic&iid=en_SgyDTHwpAdSqcltSp1vZRf612AM1OEfFOcNdmgCPdRZTQIc1vZJ38myrfG-41j-1aXtAEQQHc88KbOCHKKWeIQ%3D%3D&ppt=hp&ppn=homepage&ssid=ncuw9ar4xc0000001690536062170&qH=404e218d27fee49e'}, {'Prod_name': 'Medium 25 L Backpack SMALL 25 L DAYPACK BAGPACK FOR DAI...', 'Prod_price': '349', 'Prod_link': 'https://www.flipkart.com/plexy-small-25-l-daypack-bagpack-daily-routine-backpack/p/itm786efb9d2ee36?pid=BKPGHSBZEXWZGHW7&lid=LSTBKPGHSBZEXWZGHW7NC7OUL&marketplace=FLIPKART&q=bags&store=reh%2F4d7&srno=s_1_2&otracker=search&otracker1=search&fm=organic&iid=en_SgyDTHwpAdSqcltSp1vZRf612AM1OEfFOcNdmgCPdRYCe280Vh90cuEsbI7D8tn4hI-rFd

In [9]:
def scrape_rates_and_reviews(link):
    browser.execute_script("window.open('');")
    browser.switch_to.window(browser.window_handles[1])
    browser.get(link)

    try:
        rate_element = wait.until(EC.presence_of_element_located((By.XPATH, "//div[@class='_3LWZlK _3uSWvT']")))
        review_element = wait.until(EC.presence_of_element_located((By.XPATH, "//span[@class='_2_R_DZ']//span")))
        rate = rate_element.text
        review = review_element.text
        review = review.split()[3]
        review = review.replace(',', '')
    except:
        rate = 0
        review = 0

    browser.close()
    browser.switch_to.window(browser.window_handles[0])

    return rate, review

In [10]:
for product in products:
    product_link = product['Prod_link']
    product_rate, product_review = scrape_rates_and_reviews(product_link)
    
    product['Prod_rate'] = product_rate
    product['Prod_review'] = product_review

In [11]:
len(products)

241

In [12]:
import csv
filename = 'Flipkart_data.csv'
with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['Prod_name', 'Prod_price', 'Prod_link', 'Prod_rate', 'Prod_review', ]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for row in products:
        writer.writerow(row)

print(f"Data stored in {filename} successfully.")

Data stored in Flipkart_data.csv successfully.
